## CESM2 - LARGE ENSEMBLE (LENS2)

#### by Mauricio Rocha and Dr. Gustavo Marques

#### The objective of this notebook is to cut out the South Atlantic region, but keeping a part of the Tropical Atlantic so that the Intertropical Convergence Zone (ITCZ) region is contemplated.
#### The area we selected is 60°S-20°N and 70°W-20°E.

## Imports

In [1]:
import intake
import intake_esm
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import numpy as np
import fsspec
import cmocean
import cartopy
import cartopy.feature as cfeature
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
import pop_tools
import sys
from distributed import Client
from ncar_jobqueue import NCARCluster
sys.path.append('/glade/u/home/mauricio/south_atlantic_heat_balance/functions')
import util
from cartopy.util import add_cyclic_point
from misc import get_ij

/glade/u/apps/opt/conda/envs/npl/lib/python3.7/site-packages/dask_jobqueue/core.py:20: FutureWarning: tmpfile is deprecated and will be removed in a future release. Please use dask.utils.tmpfile instead.
  from distributed.utils import tmpfile


## Improve the workflow using clusters 

In [2]:
cluster = NCARCluster(cores=2, # The number of cores you want
                      processes=1, # How many processes
                      resource_spec='select=1:ncpus=1:mem=30GB') # Specify resources
cluster.scale(15) # workers
client = Client(cluster)
client
# Just to you know, on the resource_spec, if the memory is too low, Dask doesn't work for that kind of data. 

/glade/u/apps/opt/conda/envs/npl/lib/python3.7/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 46009 instead
  f"Port {expected} is already in use.\n"


Connection method: Cluster object,Cluster type: dask_jobqueue.PBSCluster
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/mauricio/mrocha/proxy/46009/status,
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/mauricio/mrocha/proxy/46009/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.12.206.63:46281,Workers: 0
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/mauricio/mrocha/proxy/46009/status,Total threads: 0
Started: Just now,Total memory: 0 B


## Data Ingest

#### Path

In [3]:
#%%time
catalog = intake.open_esm_datastore(
    '/glade/collections/cmip/catalog/intake-esm-datastore/catalogs/glade-cesm2-le.json'
)

/glade/u/home/mauricio/.local/lib/python3.7/site-packages/intake_esm/core.py:84: DtypeWarning: Columns (5,6,9) have mixed types.Specify dtype option on import or set low_memory=False.
  self._df, self.catalog_file = _fetch_catalog(self.esmcol_data, esmcol_obj, csv_kwargs)


In [4]:
catalog.df

,component,stream,case,member_id,variable,start_time,end_time,time_range,long_name,units,vertical_levels,frequency,path,experiment,forcing_variant,cesm_member_id,control_branch_year,cmip_experiment_id
0,atm,cam.h1,b.e21.BHISTcmip6.f09_g17.LE2-1001.001,r1i1001p1f1,ACTNI,1850-01-01,1859-12-31,18500101-18591231,Average Cloud Top ice number,m-3,1.0,day_1,/glade/campaign/cgd/cesm/CESM2-LE/timeseries/a...,historical,cmip6,1001.001,1001,CESM2_historical_r1i1001p1f1
1,atm,cam.h1,b.e21.BHISTcmip6.f09_g17.LE2-1001.001,r1i1001p1f1,ACTNI,1860-01-01,1869-12-31,18600101-18691231,Average Cloud Top ice number,m-3,1.0,day_1,/glade/campaign/cgd/cesm/CESM2-LE/timeseries/a...,historical,cmip6,1001.001,1001,CESM2_historical_r1i1001p1f1
2,atm,cam.h1,b.e21.BHISTcmip6.f09_g17.LE2-1001.001,r1i1001p1f1,ACTNI,1870-01-01,1879-12-31,18700101-18791231,Average Cloud Top ice number,m-3,1.0,day_1,/glade/campaign/cgd/cesm/CESM2-LE/timeseries/a...,historical,cmip6,1001.001,1001,CESM2_historical_r1i1001p1f1
3,atm,cam.h1,b.e21.BHISTcmip6.f09_g17.LE2-1001.001,r1i1001p1f1,ACTNI,1880-01-01,1889-12-31,18800101-18891231,Average Cloud Top ice number,m-3,1.0,day_1,/glade/campaign/cgd/cesm/CESM2-LE/timeseries/a...,historical,cmip6,1001.001,1001,CESM2_historical_r1i1001p1f1
4,atm,cam.h1,b.e21.BHISTcmip6.f09_g17.LE2-1001.001,r1i1001p1f1,ACTNI,1890-01-01,1899-12-31,18900101-18991231,Average Cloud Top ice number,m-3,1.0,day_1,/glade/campaign/cgd/cesm/CESM2-LE/timeseries/a...,historical,cmip6,1001.001,1001,CESM2_historical_r1i1001p1f1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5359838,rof,mosart.h0,b.e21.BSSP370smbb.f09_g17.LE2-1301.018,r18i1301p1f2,TOTAL_DISCHARGE_TO_OCEAN_LIQ,2075-01,2084-12,207501-208412,MOSART total discharge into ocean: LIQ,m3/s,1.0,month_1,/glade/campaign/cgd/cesm/CESM2-LE/timeseries/r...,ssp370,smbb,1301.018,1301,CESM2_ssp370_r18i1301p1f2
5359839,rof,mosart.h0,b.e21.BSSP370smbb.f09_g17.LE2-1301.018,r18i1301p1f2,TOTAL_DISCHARGE_TO_OCEAN_LIQ,2085-01,2094-12,208501-209412,MOSART total discharge into ocean: LIQ,m3/s,1.0,month_1,/glade/campaign/cgd/cesm/CESM2-LE/timeseries/r...,ssp370,smbb,1301.018,1301,CESM2_ssp370_r18i1301p1f2
5359840,rof,mosart.h0,b.e21.BSSP370smbb.f09_g17.LE2-1301.018,r18i1301p1f2,TOTAL_DISCHARGE_TO_OCEAN_LIQ,2095-01,2100-12,209501-210012,MOSART total discharge into ocean: LIQ,m3/s,1.0,month_1,/glade/campaign/cgd/cesm/CESM2-LE/timeseries/r...,ssp370,smbb,1301.018,1301,CESM2_ssp370_r18i1301p1f2
5359841,rof,mosart.h0,b.e21.BSSP370smbb.f09_g17.LE2-1301.019,r19i1301p1f2,TOTAL_DISCHARGE_TO_OCEAN_LIQ,2015-01,2024-12,201501-202412,MOSART total discharge into ocean: LIQ,m3/s,1.0,month_1,/glade/campaign/cgd/cesm/CESM2-LE/timeseries/r...,ssp370,smbb,1301.019,1301,CESM2_ssp370_r19i1301p1f2


#### How does the variable look like? 

In [5]:
cat_subset = catalog.search(component='ocn',
                            variable='SST',
                            frequency='day_1')
#                           frequency='day_1').df.variable.unique() # Here, you can see all the variables available for the frequency and for the component specified. 

In [6]:
%%time
dset_dict_raw = cat_subset.to_dataset_dict()


--> The keys in the returned dictionary of datasets are constructed as follows:
	'component.experiment.stream.forcing_variant.variable'


CPU times: user 1min 50s, sys: 3.54 s, total: 1min 54s
Wall time: 2min 27s


In [7]:
ds = dset_dict_raw['ocn.historical.pop.h.nday1.cmip6.SST']   # daily

In [8]:
%%time
SST_mean=ds.SST.isel(member_id=slice(0,5)).mean(dim="member_id")[1,:,:].compute()
SST_mean

KilledWorker: ("('mean_combine-partial-76641b846469d068c8f5e9a83ff10ab4', 0, 0, 0, 0)", <WorkerState 'tcp://10.12.206.17:40306', name: PBSCluster-9, status: closed, memory: 0, processing: 1>)

##### If you choose the ocean component, you will need to import the POP grid (bellow). For the other components, you can use the emsemble's own grid. 

#### Import the POP grid
##### In ds, TLONG and TLAT have missing values (NaNs), so we need to override them with the values from pop_grid, which does not have missing values

In [ ]:
# Read the pop 1 deg grid from pop_tools
# We will use variables TLONG and TLAT
pop_grid = pop_tools.get_grid('POP_gx1v7')
ds['TLONG'] = pop_grid.TLONG   # Longitud
ds['TLAT'] = pop_grid.TLAT     # Latitudes

In [ ]:
%%time
plt.figure(figsize=(10,6));
ax = plt.axes(projection=ccrs.Robinson());
pc = ds.SST.isel(time=0, member_id=0).plot.pcolormesh(ax=ax,
                                    transform=ccrs.PlateCarree(),
                                    cmap=cmocean.cm.thermal,
                                    x='TLONG',
                                    y='TLAT',
                                    vmin=-2,
                                    vmax=32,
                                    cbar_kwargs={"orientation": "horizontal"})                                    
ax.gridlines(draw_labels=True);
ax.coastlines()
ax.gridlines()

In [ ]:
#dsp = util.pop_add_cyclic(ds)
#dsp

#### Centralize the South Atlantic 

In [ ]:
pop_grid

In [ ]:
#i, j = get_ij(90, -22, pop_grid) 90 is lon (_i), -22 is lat (_j)
sw_i, sw_j = get_ij(-70, -60, pop_grid) # southwest
se_i, se_j = get_ij(20, -60, pop_grid) # southeast
nw_i, nw_j = get_ij(0, 20, pop_grid) # northwest
ne_i, ne_j = get_ij(320, 20, pop_grid) # northeast

print('Southwest Edge: sw_i = {}, sw_j = {}'.format(sw_i,sw_j))
print('Southeast Edge: se_i = {}, se_j = {}'.format(se_i,se_j))
print('Northwest Edge: nw_i = {}, nw_j = {}'.format(nw_i,nw_j))
print('Northeast Edge: ne_i = {}, ne_j = {}'.format(ne_i,ne_j))

# number of grid points to expand over all directions
#n = 50
#print('Region is defined by i = {},{}, j = {},{}'.format(i-n,i+n,j-n,j+n))

#### Extract area and define start and end dates

In [ ]:
area = pop_grid.TAREA.isel(nlon = slice(nw_i,ne_i), 
                           nlat = slice(sw_j,ne_j))
start = "1960-01-01" # first time
end   = "2015-01-01" # last time
area

In [ ]:
ds_remapped = ds.SST.isel(nlon = slice(nw_i,ne_i), nlat = slice(sw_j,ne_j),
                            ).sel(time = slice(start,end)).weighted(area).mean(dim=['time']).load()

In [ ]:
ds_remapped

In [ ]:
#ds_remapped.isel(member_id = 10).plot()

In [ ]:
plt.figure(figsize=(10,6));
ax = plt.axes(projection=ccrs.Robinson());
pc = ds_remapped.isel(member_id=0).plot.pcolormesh(ax=ax,
                                    transform=ccrs.PlateCarree(),
                                    cmap=cmocean.cm.thermal,
                                    x='TLONG',
                                    y='TLAT',
                                    vmin=-2,
                                    vmax=32,
                                    cbar_kwargs={"orientation": "horizontal"})                                    
ax.gridlines(draw_labels=True);
ax.coastlines()
ax.gridlines()